# Demo 1 : MD export with OpenFF
Assuming you've walked through the [previous demos on polymer system preparation](../2-preparation/2.0-index.ipynb), you'll have in hand a polymer system that you wish to simulate

The OpenFF ecosystem includes many excellent parameterization and molecular dynamics (MD) export tools  
which we show how to use here, alongside `polymerist` of course, to prepare input files for your MD engine of choice

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

from pathlib import Path
from polymerist.genutils.fileutils.pathutils import is_empty, assemble_path


EXAMPLE_DIR = Path('prepared_system_examples')
assert EXAMPLE_DIR.exists() and not is_empty(EXAMPLE_DIR)

OUTPUT_DIR = Path('scratch_MD') # dummy directory for writing without tampering with example inputs
OUTPUT_DIR.mkdir(exist_ok=True)

## Loading a prepared polymer system

As before, we provide you some example systems which we [prepared earlier](https://tvtropes.org/pmwiki/pmwiki.php/Main/OneIPreparedEarlier), namely:
* a single poly(N-isopropylacrylamide) (PNIPAAm) in vacuum: exports fastest and avoids complications with TIP3P water constraints
* a single PNIPAAm in a box of water: may have issues with LAMMPS export (per )
* a loosely-packed melt of PNIPAAm and poly(isphenol A carbonate) (PBPA) chains solvated in water, as generated in the [previous demo series](../2-preparation/2.3-melt_packing_and_solvation.ipynb):  
 this is a big system and will take a while to load!

In [ ]:
from polymerist.mdtools.openfftools.topology import topology_from_sdf


prepared_polymer_system_sdf = assemble_path(EXAMPLE_DIR, 'PNIPAAm_prepared', 'sdf')
# prepared_polymer_system_sdf = assemble_path(EXAMPLE_DIR, 'PNIPAAm_solv_water_TIP3P', 'sdf')
# prepared_polymer_system_sdf = assemble_path(EXAMPLE_DIR, 'PNIPAAm_PBPA_3x3x3_solv_water_TIP3P', 'sdf') # NOTE: this is a big Topology; it will take a minute or two to load!
polymer_name : str = prepared_polymer_system_sdf.stem
assert prepared_polymer_system_sdf.exists()

polymer_topology = topology_from_sdf(prepared_polymer_system_sdf)
polymer_topology.visualize()

NGLWidget()

### Identifying unique molecules in the system
Though the polymer topology may contain many individual molecules, it will often not contain many _distinct_ ones;  
Knowing the identities, and more importantly, the partial charges of these molecules, is important for the ensuing force field parameterization step to proceed swiftly,  
especially for systems with large polymers which would otherwise grind the automatic AM1-BCC charge assignment to a screeching halt

These unique molecules can be provided manually or obtained automatically, as show here

In [10]:
unique_mols = []
for offmol in polymer_topology.unique_molecules:
    print(offmol)
    assert offmol.partial_charges is not None
    unique_mols.append(offmol)

Molecule with name 'PNIPAAm' with Hill formula 'C162H299N27O27'
Molecule with name 'water_TIP3P' and SMILES '[H]O[H]'


### Taking inventory of the OpenFF Toolkit
OpenFF ships with many force fields and toolkits, which are often tricky to keep track of; `polymerist` provides you with convenient  
inventories of these and more, dynamically updated by what you've installed in your environment, to help you keep track of the expansive toolkit

In [11]:
from polymerist.mdtools.openfftools import (
    available_force_fields_summary,
    FF_PATH_REGISTRY, 
    ALL_IMPORTABLE_TKWRAPPERS,
    TOOLKITS_BY_CHARGE_METHOD,
    # these are just some examples; there are many registries built for you at import time
)

print(TOOLKITS_BY_CHARGE_METHOD['gasteiger'])
print(FF_PATH_REGISTRY.keys())
print(available_force_fields_summary())

[<class 'openff.toolkit.utils.rdkit_wrapper.RDKitToolkitWrapper'>, <class 'openff.toolkit.utils.ambertools_wrapper.AmberToolsToolkitWrapper'>, <class 'openff.toolkit.utils.openeye_wrapper.OpenEyeToolkitWrapper'>]
dict_keys(['smirnoff99frosst', 'amber_ff_ports', 'openforcefields'])
smirnoff99frosst
├── smirnoff99Frosst-1.0.9
├── smirnoff99Frosst-1.0.1
├── smirnoff99Frosst-1.0.7
├── smirnoff99Frosst-1.0.3
├── smirnoff99Frosst-1.0.8
├── smirnoff99Frosst-1.0.2
├── smirnoff99Frosst-1.0.5
├── smirnoff99Frosst-1.0.4
├── smirnoff99Frosst-1.1.0
├── smirnoff99Frosst-1.0.6
└── smirnoff99Frosst-1.0.0
amber_ff_ports
├── ff14sb_0.0.3
├── ff14sb_off_impropers_0.0.3
├── ff14sb_0.0.1
├── ff14sb_0.0.4
├── ff14sb_0.0.2
├── ff14sb_off_impropers_0.0.1
├── ff14sb_off_impropers_0.0.2
└── ff14sb_off_impropers_0.0.4
openforcefields
├── openff_unconstrained-2.2.0
├── openff_unconstrained-2.2.0-rc1
├── spce-1.0.0
├── openff-1.0.0-RC2
├── openff-1.3.1
├── openff_unconstrained-1.1.0
├── openff_unconstrained-1.0.0


## Assigning force field parameters and creating an Interchange
See OpenFF docs on [Forcefield](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.typing.engines.smirnoff.ForceField.html) and [Interchange](https://docs.openforcefield.org/projects/interchange/en/stable/using/intro.html) for more details there. For demonstration, we use the [Sage 2.y.z force field](https://openforcefield.org/force-fields/force-fields/), which include TIP3P water parameters

In [12]:
from openff.toolkit import ForceField, Molecule
from openff.interchange import Interchange
logging.getLogger('openff.interchange.smirnoff._nonbonded').setLevel(logging.CRITICAL) # suppress spammy "Preset charges..." logs otherwise printed for EVERY atom in an Interchange

from polymerist.mdtools.openfftools.boxvectors import get_topology_bbox


forcefield = ForceField('openff-2.0.0.offxml')
# forcefield = ForceField('openff-2.2.0.offxml')
# forcefield = ForceField('openff-2.0.0.offxml', 'tip3p.offxml') # you can also combine multiple force fields
interchange = Interchange.from_smirnoff(
    topology=polymer_topology,
    force_field=forcefield,
    charge_from_molecules=unique_mols, # this is VITAL for speedy processing of polymers; omitting this will grind everythin to a halt
)
interchange.box = get_topology_bbox(polymer_topology) # this is also key for nonbonded interactions to behave correctly

INFO:openff.toolkit.typing.engines.smirnoff.parameters:Attempting to up-convert vdW section from 0.3 to 0.4
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Successfully up-converted vdW section from 0.3 to 0.4. `method="cutoff"` is now split into `periodic_method="cutoff"` and `nonperiodic_method="no-cutoff"`.
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Attempting to up-convert Electrostatics section from 0.3 to 0.4
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Successfully up-converted Electrostatics section from 0.3 to 0.4. `method="PME"` is now split into `periodic_potential="Ewald3D-ConductingBoundary"`, `nonperiodic_potential="Coulomb"`, and `exception_potential="Coulomb"`.


### Export to OpenMM
OpenMM is the best supported by both Interchange and `polymerist`, mainly due to its easy-to-automate Python API  
In ensuing examples, I'll show some of the ways `polymerist` uses this API to simplify running OpenMM simulations

In [13]:
from polymerist.mdtools.openmmtools.serialization import serialize_openmm_pdb, serialize_system


OMM_DIR = OUTPUT_DIR / 'OpenMM'
OMM_DIR.mkdir(exist_ok=True)

omm_sys = interchange.to_openmm_system(combine_nonbonded_forces=False)
omm_top = interchange.to_openmm_topology(collate=True)
omm_pos = interchange.get_positions()

serialize_system(assemble_path(OMM_DIR, polymer_name, 'xml'), system=omm_sys)
serialize_openmm_pdb(assemble_path(OMM_DIR, polymer_name, 'pdb'), topology=omm_top, positions=omm_pos)

/home/timber/miniconda3/envs/polymerist-dev/lib/python3.11/site-packages/openmm/app/pdbfile.py:347: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  positions = np.asarray(positions)


### Export to GROMACS

In [14]:
GMX_DIR = OUTPUT_DIR / 'GROMACS'
GMX_DIR.mkdir(exist_ok=True)

# NOTE: even for relatively small systems, this is exceedingly slow (https://github.com/openforcefield/openff-interchange/issues/1264)
interchange.to_gromacs(prefix=str(GMX_DIR/polymer_name)) 

/home/timber/miniconda3/envs/polymerist-dev/lib/python3.11/site-packages/openff/interchange/components/mdconfig.py:473: UserWarning: Ambiguous failure while processing constraints. Constraining h-bonds as a stopgap.
  warnings.warn(


### Export to LAMMPS

In [15]:
LMP_DIR = OUTPUT_DIR / 'LAMMPS'
LMP_DIR.mkdir(exist_ok=True)


# NOTE: Interchange.to_lammps(...) combines both of the below; we've shown them separately for clarity

# Interchange's LAMMPS writer currently doesn't support bond constraints, meaning this export will only work when using unconstrained force fields
# https://github.com/openforcefield/openff-interchange/issues/892
lmp_data_path = assemble_path(LMP_DIR, polymer_name, 'lmp')
interchange.to_lammps_datafile(lmp_data_path)

# At the time of writing, the input file writer will fail for systems containing water, citing missing constraints
# https://github.com/openforcefield/openff-interchange/issues/1141
lmp_input_path = assemble_path(LMP_DIR, polymer_name, 'in')
interchange.to_lammps_input(lmp_input_path, data_file=lmp_data_path) # NOTE: currently, this currently does NOT support constrained force fields

NotImplementedError: Found unsupported constraints case in LAMMPS input writer.